In [1]:
import json
from collections import OrderedDict
import torch
from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
from flask import Flask, jsonify, request

In [2]:
model = torch.jit.load('model.pt')

In [7]:
def preprocess(img_path = "img_129.jpg"):
    img = Image.open(img_path).convert('L')
    transform = transforms.Compose([transforms.Grayscale(), transforms.ToTensor()])
    return transform(img).unsqueeze(0)



def get_prediction(image_path):
    class_index = json.load(open('class_index.json'))
    tensor = preprocess(image_path)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = str(y_hat.item())
    return class_index[predicted_idx]

In [9]:
get_prediction('img_129.jpg')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)